In [32]:
import sys
print(sys.executable)

C:\Users\chenr\miniforge3\envs\astropy\python.exe


In [61]:
cd ..

C:\Users\chenr\OneDrive\Desktop\phys-134-proj


In [62]:
import os
rootfolder=os.getcwd()
#turn \\ -> /
rootfolder=rootfolder.replace('\\','/')
print(rootfolder)

C:/Users/chenr/OneDrive/Desktop/phys-134-proj


In [34]:
# --- IGNORE ---
#! cd C:/Users/chenr/OneDrive/Desktop/phys-134-proj

In [35]:
from astropy.samp import SAMPIntegratedClient

In [36]:
ds9 = SAMPIntegratedClient()

ds9.connect()

In [37]:
#SAMP is a messaging protocol that enables astronomy software tools to interoperate and communicate. Broadly speaking, SAMP is an abstract framework for loosely-coupled, asynchronous, RPC-like and/or event-based communication, based on a central service providing multi-directional publish/subscribe message brokering. The message semantics are extensible and use structured but weakly-typed data. For more information on SAMP, please click here.

#The samp implementation for DS9 is based on the XPA model with 2 private calls:

# ds9.get
#     Arguments
#         cmd (string) required
#         url (string) optional
#     Returned value
#         OK (samp.result map)
#             value (string) optional
#                 url (string) optional
#         ERROR (samp.error map)
#             samp.errortxt (string)
 
# ds9.set
#     Arguments
#         cmd (string) required
#         url (string) optional
#     Returned value
#         OK
#         ERROR (samp.error map)
#             samp.errortxt (string)

In [38]:
ds9.get_registered_clients()

['hub', 'c1', 'c2']

In [39]:
ds9.get_metadata('c1')

{'samp.name': 'ds9',
 'home.page': 'http://ds9.si.edu/',
 'ds9.version': '8.6',
 'samp.icon.url': 'http://ds9.si.edu/sun.png',
 'author.email': 'ds9help@cfa.harvard.edu',
 'samp.description.text': 'SAOImageDS9 is an astronomical visualization application',
 'samp.documentation.url': 'http://ds9.si.edu/doc/ref/index.html',
 'author.name': 'William Joye',
 'author.affiliation': 'Smithsonian Astrophysical Observatory'}

In [40]:
#ecall and wait:
# recipient id, mtype, timeout, params: dict of set of str 

In [41]:
# on ds9 launch, expecting 1 frame to be already loaded
#what we need to do:
# load first image
# new frame
# load second image
# ... until no more images left

In [53]:
ds9.ecall_and_wait("c1", "ds9.get", "10", cmd = "file")

{'samp.result': {'value': 'C:/Users/chenr/OneDrive/Desktop/phys-134-proj/lco_data-20251202-36/2025-11-25-h09-m06-s12-ms841-elp.fits[SCI]'},
 'samp.status': 'samp.ok'}

In [50]:
ds9.ecall_and_wait("c1","ds9.get","10",cmd="cd")

{'samp.result': {'value': 'C:/SAOImageDS9'}, 'samp.status': 'samp.ok'}

In [13]:
#get working directory

In [64]:
ds9.ecall_and_wait("c1","ds9.set","10",cmd=f"cd {rootfolder}/lco_data-20251202-36")

{'samp.result': {}, 'samp.status': 'samp.ok'}

In [65]:
ds9.ecall_and_wait("c1","ds9.get","10",cmd="cd")

{'samp.result': {'value': 'C:/Users/chenr/OneDrive/Desktop/phys-134-proj/lco_data-20251202-36'},
 'samp.status': 'samp.ok'}

In [66]:
#assumed client object instantiated as "ds9"
def clientset(cmd, client_id="c1", timeout=10):
    return ds9.ecall_and_wait(client_id, "ds9.set", str(timeout), cmd=cmd)
def clientget(cmd, client_id="c1", timeout=10):
    return ds9.ecall_and_wait(client_id, "ds9.get", str(timeout), cmd=cmd)

In [74]:
#delte all frames first
clientset(cmd="frame delete all")

{'samp.result': {}, 'samp.status': 'samp.ok'}

Start loading images

In [72]:
#get array of files in directory
file_list=os.listdir(f"{rootfolder}/lco_data-20251202-36")

# filter to only .fits files
file_list=[f for f in file_list if f.endswith('.fits') and f.startswith('2025')]
print(file_list)


['2025-11-25-h02-m00-s14-ms936-tfn.fits', '2025-11-25-h05-m00-s13-ms688-lsc.fits', '2025-11-25-h05-m36-s42-ms454-lsc.fits', '2025-11-25-h06-m06-s14-ms909-lsc.fits', '2025-11-25-h07-m45-s12-ms840-elp.fits', '2025-11-25-h08-m30-s09-ms718-elp.fits', '2025-11-25-h09-m06-s12-ms841-elp.fits', '2025-11-25-h10-m06-s14-ms367-elp.fits', '2025-11-25-h11-m06-s18-ms072-elp.fits', '2025-11-25-h12-m47-s39-ms526-ogg.fits', '2025-11-25-h13-m06-s13-ms287-ogg.fits', '2025-11-25-h14-m06-s15-ms196-ogg.fits', '2025-11-25-h23-m02-s40-ms929-tfn.fits', '2025-11-25-h23-m35-s17-ms210-cpt.fits', '2025-11-26-h00-m55-s17-ms907-tfn.fits', '2025-11-26-h01-m55-s17-ms940-tfn.fits', '2025-11-26-h02-m10-s07-ms425-tfn.fits', '2025-11-26-h03-m06-s19-ms581-tfn.fits', '2025-11-26-h04-m06-s16-ms133-tfn.fits', '2025-11-26-h05-m35-s14-ms393-lsc.fits', '2025-11-26-h06-m06-s06-ms084-lsc.fits', '2025-11-26-h07-m59-s44-ms591-elp.fits', '2025-11-26-h08-m06-s06-ms512-elp.fits', '2025-11-26-h11-m06-s14-ms445-ogg.fits', '2025-11-26-h12

In [75]:
for idx,imdir  in enumerate(file_list):
    print(f"Loading file {idx+1} of {len(file_list)}: {imdir}")
    clientset(cmd="frame new")
    clientset(cmd=f"fits {imdir}")

Loading file 1 of 36: 2025-11-25-h02-m00-s14-ms936-tfn.fits
Loading file 2 of 36: 2025-11-25-h05-m00-s13-ms688-lsc.fits
Loading file 3 of 36: 2025-11-25-h05-m36-s42-ms454-lsc.fits
Loading file 4 of 36: 2025-11-25-h06-m06-s14-ms909-lsc.fits
Loading file 5 of 36: 2025-11-25-h07-m45-s12-ms840-elp.fits
Loading file 6 of 36: 2025-11-25-h08-m30-s09-ms718-elp.fits
Loading file 7 of 36: 2025-11-25-h09-m06-s12-ms841-elp.fits
Loading file 8 of 36: 2025-11-25-h10-m06-s14-ms367-elp.fits
Loading file 9 of 36: 2025-11-25-h11-m06-s18-ms072-elp.fits
Loading file 10 of 36: 2025-11-25-h12-m47-s39-ms526-ogg.fits
Loading file 11 of 36: 2025-11-25-h13-m06-s13-ms287-ogg.fits
Loading file 12 of 36: 2025-11-25-h14-m06-s15-ms196-ogg.fits
Loading file 13 of 36: 2025-11-25-h23-m02-s40-ms929-tfn.fits
Loading file 14 of 36: 2025-11-25-h23-m35-s17-ms210-cpt.fits
Loading file 15 of 36: 2025-11-26-h00-m55-s17-ms907-tfn.fits
Loading file 16 of 36: 2025-11-26-h01-m55-s17-ms940-tfn.fits
Loading file 17 of 36: 2025-11-26

In [76]:
# apply catalog to all frames
catid="{I/340}"
for idx in range(len(file_list)):
    print(f"Applying catalog to frame {idx+1} of {len(file_list)}")
    clientset(cmd=f"frame {idx+1}")
    clientset(cmd=f"catalog cds {catid}")
    clientset(cmd="catalog filter {$Gmag<13 && $Gmag>12.2}")
    # Reference stars established
    
    clientset(cmd=f"catalog save catalogs/{file_list[idx]}.xml")
    
    #turn into regions
    clientset(cmd="catalog regions")

    #close catalogue
    clientset(cmd="catalog clear")
    clientset(cmd="catalog close")
    
    #select all regions
    clientset(cmd="region select all")
    # centroid parameters
    clientset(cmd="region centroid radius 15")
    clientset(cmd="region centroid iteration 10")
    #find centroid
    clientset(cmd="region centroid")
    
    #save centroids
    clientset(cmd=f"region save select regions/{file_list[idx]}.reg")
    
    #delete all regions for next frame
    clientset(cmd="region delete all")
        

Applying catalog to frame 1 of 36
Applying catalog to frame 2 of 36
Applying catalog to frame 3 of 36
Applying catalog to frame 4 of 36
Applying catalog to frame 5 of 36
Applying catalog to frame 6 of 36
Applying catalog to frame 7 of 36
Applying catalog to frame 8 of 36
Applying catalog to frame 9 of 36
Applying catalog to frame 10 of 36
Applying catalog to frame 11 of 36
Applying catalog to frame 12 of 36
Applying catalog to frame 13 of 36
Applying catalog to frame 14 of 36
Applying catalog to frame 15 of 36
Applying catalog to frame 16 of 36
Applying catalog to frame 17 of 36
Applying catalog to frame 18 of 36
Applying catalog to frame 19 of 36
Applying catalog to frame 20 of 36
Applying catalog to frame 21 of 36
Applying catalog to frame 22 of 36
Applying catalog to frame 23 of 36
Applying catalog to frame 24 of 36
Applying catalog to frame 25 of 36
Applying catalog to frame 26 of 36
Applying catalog to frame 27 of 36
Applying catalog to frame 28 of 36
Applying catalog to frame 29 

In [78]:
# we are done with ds9
ds9.disconnect()

# Reference Star Identification

The cat stars and regions are index-aligned

### Regions

In [79]:
#python working directory
import os
rootfolder=os.getcwd().replace('\\','/')

#change working directory to folder with data
os.chdir(f"{rootfolder}/lco_data-20251202-36/regions/")
os.getcwd().replace('\\','/')

'C:/Users/chenr/OneDrive/Desktop/phys-134-proj/lco_data-20251202-36/regions'

In [80]:
#file format:
# ```
#1 # Region file format: DS9 version 4.1
#2 global color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1
#3 physical
#4 point(1040.7514,115.65428) # point=circle
#5 point(1241.4606,105.02064) # point=circle
#6 point(674.4981,326.19546) # point=circle
#7 point(515.31476,308.61345) # point=circle
#8 point(967.08735,270.63355) # point=circle
#...
# ```

#store like this:

# root
#  filename (no extensions, strip[0] '.')
#     x: []
#     y: []

# we need to extract the pixel coordinates from each file

import re

region_data={}
for rf in region_files:
    print(f"Processing region file: {rf}")
    with open(rf,'r') as f:
        lines=f.readlines()
        x_coords=[]
        y_coords=[]
        for line in lines:
            #match point(x,y)
            match=re.match(r'point\(([-+]?\d*\.\d+|\d+),([-+]?\d*\.\d+|\d+)\)', line)
            if match:
                x=float(match.group(1))
                y=float(match.group(2))
                x_coords.append(x)
                y_coords.append(y)
        #store in dictionary
        key=rf.rsplit('.',1)[0]  #strip extension
        region_data[key]={'x':x_coords, 'y':y_coords}

Processing region file: 2025-11-25-h02-m00-s14-ms936-tfn.fits.reg
Processing region file: 2025-11-25-h05-m00-s13-ms688-lsc.fits.reg
Processing region file: 2025-11-25-h05-m36-s42-ms454-lsc.fits.reg
Processing region file: 2025-11-25-h06-m06-s14-ms909-lsc.fits.reg
Processing region file: 2025-11-25-h07-m45-s12-ms840-elp.fits.reg
Processing region file: 2025-11-25-h08-m30-s09-ms718-elp.fits.reg
Processing region file: 2025-11-25-h09-m06-s12-ms841-elp.fits.reg
Processing region file: 2025-11-25-h10-m06-s14-ms367-elp.fits.reg
Processing region file: 2025-11-25-h11-m06-s18-ms072-elp.fits.reg
Processing region file: 2025-11-25-h12-m47-s39-ms526-ogg.fits.reg
Processing region file: 2025-11-25-h13-m06-s13-ms287-ogg.fits.reg
Processing region file: 2025-11-25-h14-m06-s15-ms196-ogg.fits.reg
Processing region file: 2025-11-25-h23-m02-s40-ms929-tfn.fits.reg
Processing region file: 2025-11-25-h23-m35-s17-ms210-cpt.fits.reg
Processing region file: 2025-11-26-h00-m55-s17-ms907-tfn.fits.reg
Processing

In [21]:
region_data[list(region_data.keys())[0]]  

{'x': [1040.7514,
  1241.4606,
  674.4981,
  515.31476,
  967.08735,
  618.71406,
  640.87394,
  292.04352,
  137.36356,
  433.49042,
  283.50617,
  79.425203,
  1844.1393,
  2021.7744,
  1922.9335,
  1637.4897,
  1821.7388,
  1813.2322,
  889.37272,
  1070.5171,
  1197.6024,
  1142.0139,
  1158.0403,
  976.50191,
  1299.3982,
  2335.6437,
  2154.8672,
  1778.2506,
  1548.8076,
  1566.3322,
  1462.0771,
  1701.1373,
  435.65532,
  361.58235,
  572.86304,
  845.9567,
  931.70197,
  868.3275,
  767.30246,
  1063.187,
  1131.5402,
  746.4488,
  1324.1962,
  1285.4498],
 'y': [115.65428,
  105.02064,
  326.19546,
  308.61345,
  270.63355,
  699.36388,
  704.06969,
  761.61582,
  826.60628,
  790.26934,
  1152.3433,
  1232.1294,
  379.90335,
  578.41979,
  660.68761,
  700.40258,
  745.10545,
  1064.1231,
  642.35046,
  853.07592,
  1204.7118,
  1280.7725,
  1484.9646,
  1442.9895,
  1660.9414,
  1128.4766,
  1593.4748,
  1785.8328,
  1687.114,
  1860.1909,
  2149.205,
  2155.8444,
  1590.4

In [88]:
region_data

{'2025-11-25-h02-m00-s14-ms936-tfn.fits': {'x': [674.51008,
   515.45897,
   618.67282,
   137.7245,
   2021.3085,
   1637.122,
   1299.9491,
   2154.6617,
   1548.8365,
   1462.3786,
   1700.4093,
   435.45068,
   361.83888,
   573.49969,
   1324.5271,
   1285.7478],
  'y': [325.71172,
   308.99125,
   699.70433,
   826.61105,
   578.00791,
   700.31366,
   1661.0504,
   1593.8816,
   1686.4079,
   2149.6571,
   2155.6244,
   1590.5307,
   1741.6414,
   1763.3525,
   2172.8186,
   2339.1397]},
 '2025-11-25-h05-m00-s13-ms688-lsc.fits': {'x': [669.8357,
   508.78492,
   612.81842,
   128.88614,
   2024.0029,
   1637.4491,
   1296.4271,
   2156.8919,
   1547.0131,
   1459.3625,
   1698.9329,
   427.27715,
   353.71118,
   565.47001,
   1320.8434,
   1281.793],
  'y': [318.54021,
   301.40698,
   694.58834,
   821.14283,
   575.56554,
   696.37803,
   1662.1788,
   1595.7014,
   1688.683,
   2153.2121,
   2159.3934,
   1590.2609,
   1741.3498,
   1764.1286,
   2176.0201,
   2343.2675]},
 

In [85]:
#go up 1 directory
os.chdir('..')

In [23]:
#gwd
os.getcwd()

'C:\\Users\\chenr\\OneDrive\\Desktop\\phys-134-proj\\lco_data-20251202-36'

### Catalogs

In [ ]:
os.chdir("catalogs")

In [87]:
os.getcwd()

'C:\\Users\\chenr\\OneDrive\\Desktop\\phys-134-proj\\lco_data-20251202-36\\catalogs'

In [91]:
file_list=os.listdir(".")

# filter to only .fits files
file_list=[f for f in file_list if f.endswith('.xml')]
print(file_list)


['2025-11-25-h02-m00-s14-ms936-tfn.fits.xml', '2025-11-25-h05-m00-s13-ms688-lsc.fits.xml', '2025-11-25-h05-m36-s42-ms454-lsc.fits.xml', '2025-11-25-h06-m06-s14-ms909-lsc.fits.xml', '2025-11-25-h07-m45-s12-ms840-elp.fits.xml', '2025-11-25-h08-m30-s09-ms718-elp.fits.xml', '2025-11-25-h09-m06-s12-ms841-elp.fits.xml', '2025-11-25-h10-m06-s14-ms367-elp.fits.xml', '2025-11-25-h11-m06-s18-ms072-elp.fits.xml', '2025-11-25-h12-m47-s39-ms526-ogg.fits.xml', '2025-11-25-h13-m06-s13-ms287-ogg.fits.xml', '2025-11-25-h14-m06-s15-ms196-ogg.fits.xml', '2025-11-25-h23-m02-s40-ms929-tfn.fits.xml', '2025-11-25-h23-m35-s17-ms210-cpt.fits.xml', '2025-11-26-h00-m55-s17-ms907-tfn.fits.xml', '2025-11-26-h01-m55-s17-ms940-tfn.fits.xml', '2025-11-26-h02-m10-s07-ms425-tfn.fits.xml', '2025-11-26-h03-m06-s19-ms581-tfn.fits.xml', '2025-11-26-h04-m06-s16-ms133-tfn.fits.xml', '2025-11-26-h05-m35-s14-ms393-lsc.fits.xml', '2025-11-26-h06-m06-s06-ms084-lsc.fits.xml', '2025-11-26-h07-m59-s44-ms591-elp.fits.xml', '2025-11-

In [92]:
# load all xml catalog files

#per file: data is sandwiched between <TABLEDATA> and </TABLEDATA>
#each line is like <TR><TD>077.969011446</TD><TD>+18.033992941</TD><TD>077.9690121</TD><TD>+18.0339919</TD><TD>3395463877063938432</TD><TD>77.9690228</TD><TD>0.2</TD><TD>18.0339756</TD><TD>0.2</TD><TD>2</TD><TD>4</TD><TD>2000.915</TD><TD>2.6</TD><TD>1.3</TD><TD>-4.2</TD><TD>1.2</TD><TD>12.973</TD><TD>13.243</TD><TD>13.100</TD><TD>12.113</TD><TD>11.951</TD><TD>11.864</TD></TR>
# we only want the first 2 columns: RA, Dec
catalog_data={}
for cf in file_list:
    print(f"Processing catalog file: {cf}")
    with open(cf,'r') as f:
        lines=f.readlines()
        ra_list=[]
        dec_list=[]
        in_table=False
        for line in lines:
            if '<TABLEDATA>' in line:
                in_table=True
                continue
            if '</TABLEDATA>' in line:
                in_table=False
                continue
            if in_table:
                #extract RA and Dec from line
                match=re.findall(r'<TD>(.*?)</TD>', line)
                if match and len(match)>=2:
                    ra=float(match[0])
                    dec=float(match[1])
                    ra_list.append(ra)
                    dec_list.append(dec)
        #store in dictionary
        key=cf.rsplit('.',1)[0]  #strip extension
        catalog_data[key]={'RA':ra_list, 'Dec':dec_list}


Processing catalog file: 2025-11-25-h02-m00-s14-ms936-tfn.fits.xml
Processing catalog file: 2025-11-25-h05-m00-s13-ms688-lsc.fits.xml
Processing catalog file: 2025-11-25-h05-m36-s42-ms454-lsc.fits.xml
Processing catalog file: 2025-11-25-h06-m06-s14-ms909-lsc.fits.xml
Processing catalog file: 2025-11-25-h07-m45-s12-ms840-elp.fits.xml
Processing catalog file: 2025-11-25-h08-m30-s09-ms718-elp.fits.xml
Processing catalog file: 2025-11-25-h09-m06-s12-ms841-elp.fits.xml
Processing catalog file: 2025-11-25-h10-m06-s14-ms367-elp.fits.xml
Processing catalog file: 2025-11-25-h11-m06-s18-ms072-elp.fits.xml
Processing catalog file: 2025-11-25-h12-m47-s39-ms526-ogg.fits.xml
Processing catalog file: 2025-11-25-h13-m06-s13-ms287-ogg.fits.xml
Processing catalog file: 2025-11-25-h14-m06-s15-ms196-ogg.fits.xml
Processing catalog file: 2025-11-25-h23-m02-s40-ms929-tfn.fits.xml
Processing catalog file: 2025-11-25-h23-m35-s17-ms210-cpt.fits.xml
Processing catalog file: 2025-11-26-h00-m55-s17-ms907-tfn.fits

In [93]:
catalog_data[list(catalog_data.keys())[0]]

{'RA': [77.969011446,
  78.003830657,
  77.981074191,
  78.086118951,
  77.674916178,
  77.758510017,
  77.832281089,
  77.645501922,
  77.777891596,
  77.796650407,
  77.744530987,
  78.021323243,
  78.037352533,
  77.991153734,
  77.826763,
  77.835210497],
 'Dec': [18.033992941,
  18.030510528,
  18.111590059,
  18.138177951,
  18.086108028,
  18.111679268,
  18.311005504,
  18.296757124,
  18.316311176,
  18.412263872,
  18.41350691,
  18.29639713,
  18.327676943,
  18.332386485,
  18.41757886,
  18.451670982]}

In [111]:
#catalog_data is
# {#   filename:
#       RA: []
#       Dec: []
#  } #whose indexes of RA and DEc correspond to those in region_data

#region_data is
# {#   filename:
#       x: []
#       y: []
#  } #whose indexes of x and y correspond to those in catalog_data

#merge both into single data structure
merged_data={}
for key in catalog_data.keys():
    merged_data[key]={
        'icrs': {
            'ra': catalog_data[key]['RA'],
            'dec': catalog_data[key]['Dec']
        },
        'image': {
            'x': region_data[key]['x'],
            'y': region_data[key]['y']
        }
    }

Data is balanced!

In [118]:
# we know the data is balanced & ordered within an observation

# but we need to make sure that the i_th index of icrs:ra,dec and image:x,y correspond to the same image.
#luckily, icrs: ra,dec are funcationally primary keys.

#lets first check to make sure that the icrs:ra,dec are the SAME across all files

#take the first file's ra,dec as reference
ref_key=list(merged_data.keys())[0]
ref_ra = merged_data[ref_key]['icrs']['ra']
ref_dec = merged_data[ref_key]['icrs']['dec']

for key in merged_data.keys():
    ra_list = merged_data[key]['icrs']['ra']
    dec_list = merged_data[key]['icrs']['dec']
    if ra_list != ref_ra or dec_list != ref_dec:
        print(f"Mismatch found in file: {key}")

Data is ordered between observations!

In [105]:
cd ..

C:\Users\chenr\OneDrive\Desktop\phys-134-proj\lco_data-20251202-36


In [119]:
#save merged data to pickle
import pickle
with open('merged_data.pkl','wb') as f:
    pickle.dump(merged_data,f)

In [124]:
merged_data[list(merged_data.keys())[0]]['image']['x']

[674.51008,
 515.45897,
 618.67282,
 137.7245,
 2021.3085,
 1637.122,
 1299.9491,
 2154.6617,
 1548.8365,
 1462.3786,
 1700.4093,
 435.45068,
 361.83888,
 573.49969,
 1324.5271,
 1285.7478]